In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset, Subset
from torchvision import transforms, datasets, models
import numpy as np

# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터 전처리 설정
basic_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))
])

augment1 = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))
])

augment2 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(45),
    transforms.RandomHorizontalFlip(p=0.7),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))
])

# 데이터셋 로드 및 결합
trainset_path = "C:/Users/USER/Desktop/Semester2/train_data"
user_dataset = datasets.ImageFolder(root=trainset_path, transform=basic_transform)

cifar10 = datasets.CIFAR10(root="C:/Users/USER/Desktop/Semester2/CIFAR10", train=True, download=False, transform=basic_transform)
cifar10_dataset1 = datasets.CIFAR10(root="C:/Users/USER/Desktop/Semester2/CIFAR10", train=True, download=False, transform=augment1)
cifar10_dataset2 = datasets.CIFAR10(root="C:/Users/USER/Desktop/Semester2/CIFAR10", train=True, download=False, transform=augment2)

# 랜덤하게 10,000장씩 선택
num_samples = 10000
indices1 = np.random.choice(len(cifar10_dataset1), num_samples, replace=False)
indices2 = np.random.choice(len(cifar10_dataset2), num_samples, replace=False)
cifar10_dataset1 = Subset(cifar10_dataset1, indices1)
cifar10_dataset2 = Subset(cifar10_dataset2, indices2)

# 데이터셋 결합
combined_dataset = ConcatDataset([user_dataset, cifar10, cifar10_dataset1, cifar10_dataset2])

# 데이터 로더 설정
train_loader = DataLoader(combined_dataset, batch_size=32, shuffle=True)

# 모델 정의 및 학습 설정
model = models.efficientnet_b0(pretrained=False)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 10)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, epochs=25, steps_per_epoch=len(train_loader))

num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    correct_train, total_train = 0, 0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        # 모델 학습
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        _, preds = outputs.max(1)
        correct_train += preds.eq(targets).sum().item()
        total_train += targets.size(0)

    train_acc = 100 * correct_train / total_train
    print(f"Epoch [{epoch + 1}/{num_epochs}], Train Acc: {train_acc:.2f}%, Loss: {loss.item():.4f}, LR: {scheduler.get_last_lr()[0]:.6f}")

# 모델 이름 생성 및 저장
model_name = "EfficientNet-B0_aug1.pth"
torch.save(model.state_dict(), f"C:/Users/USER/Desktop/Semester2/{model_name}")
print(f"최종 모델이 '{model_name}'로 저장되었습니다.")

In [ ]:
import os
# 9. CIFAR-10 클래스 이름 정의
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# 10. 테스트 데이터 로드 및 예측 수행
test_data_path = "C:/Users/USER/Desktop/Semester2/test_data"
test_filenames = sorted([f for f in os.listdir(test_data_path) if f.endswith('.png')])
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))
])

# 11. 예측 결과 저장을 위한 리스트 초기화
results = []

# 12. 예측 수행
model.eval()  # 평가 모드로 설정
with torch.no_grad():
    for filename in test_filenames:
        # 이미지 불러오기 및 전처리
        img_path = os.path.join(test_data_path, filename)
        image = Image.open(img_path).convert("RGB")
        image = test_transform(image).unsqueeze(0).to(device)

        # 모델 예측 수행
        output = model(image)
        _, predicted = output.max(1)
        label = class_names[predicted.item()]

        # 결과를 리스트에 추가
        results.append({"filename": filename, "label": label})

# 13. 예측 결과를 DataFrame으로 변환 후 CSV 파일로 저장
df = pd.DataFrame(results)
output_csv_path = "C:/Users/USER/Desktop/Semester2/test_predictions.csv"
df.to_csv(output_csv_path, index=False)

print(f"예측 결과가 {output_csv_path}에 저장되었습니다.")